## In this notebook, a dataframe will be created to store API info from some airports and its codes, then upload the gathered data to MySQL, where a database has been created beforehand.

In [1]:
#Importing the necessary libraries

import requests
import pandas as pd

In [2]:
# The lat and lon from the chosen airports is displayed below in comments, so they can be later stored in a list

# madrid_lat = '40.4983'
# madrid_lon = '3.5676'
# barcelona_lat = '41.3833'
# barcelon_lon = '2.183333'
# malaga_lat = '36.7194'
# malaga_lon = '-4.420000'
# berlin_lat = '52.5200'
# berlin_lon = '13.405000'
# hamburg_lat = '53.5500'
# hamburg_lon = '10.000000'
# london_lat = '51.5072'
# london_lon = '0.127500'

lat_airport = ['40.4983','41.2974','36.7194','52.5200','53.5500','51.5072']
lon_airport = ['-3.5676','2.08329','-4.420000','13.405000','10.000000','0.127500']

# Here a function has been created that takes lat and lon as arguments, then inside the function, a for loop is also
# created to go through the API and collection the required info. Once the info is collected, it will be appended to 
# the empty list.

def lat_lon_airports(lat,lon):
    
    lat_lon_cities_air = []
    for i in range(len(lat)):
      url = f"https://aerodatabox.p.rapidapi.com/airports/search/location/{lat[i]}/{lon[i]}/km/10/16"

      querystring = {"withFlightInfoOnly":"true"}

      headers = {
        "X-RapidAPI-Key": "My_API_key",
        "X-RapidAPI-Host": "aerodatabox.p.rapidapi.com"
      }

      response = requests.request("GET", url, headers=headers, params=querystring)

      lat_lon_cities_air.append(pd.json_normalize(response.json()['items']))

    return pd.concat(lat_lon_cities_air, ignore_index=True)

In [3]:
# The function and its parameters are stored in a variable
airports = lat_lon_airports(lat_airport,lon_airport)

In [4]:
# Dropping the columns that are not necessary
airports.drop(['iata', 'name', 'shortName','countryCode','location.lat','location.lon'], axis=1, inplace=True)

In [5]:
# Creating two columns for the ICAO code and city names.
airports.columns = ['icao','city_name']

In [6]:
# Inserting a new column called city ID to give each city a distinct number
airports.insert(0, "city_id", [1, 2, 3, 4, 5, 6], True)

In [7]:
airports

,city_id,icao,city_name
0,1,LEMD,Madrid
1,2,LEBL,Barcelona
2,3,LEMG,Málaga
3,4,EDDT,Berlin
4,5,EDDH,Hamburg
5,6,EGLC,London


In [8]:
# Checking that the datatypes for the dataframe are correct
airports.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 3 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   city_id    6 non-null      int64 
 1   icao       6 non-null      object
 2   city_name  6 non-null      object
dtypes: int64(1), object(2)
memory usage: 272.0+ bytes


In [49]:
# Importing the libraries to upload the data to MySQL
import sqlalchemy
import pymysql

In [50]:
schema="GansWeatherDB"
host="127.0.0.1"
user="My_user"
password="My_pass"
port=3306
con = f'mysql+pymysql://{user}:{password}@{host}:{port}/{schema}'

In [51]:
airports.dropna().to_sql('airports', if_exists='append', con=con, index=False)

In [42]:
airports.to_csv('airports.csv')